# 02 — Les 5 vérifications à faire juste après l’ouverture d’un CSV (pandas)

Ce notebook formalise un **rituel en 5 étapes** à appliquer immédiatement après l’import d’un fichier CSV.

## Objectif

Comprendre rapidement la structure du dataset et détecter les problèmes classiques (types, valeurs manquantes, doublons).

In [4]:
import pandas as pd
from pathlib import Path

## Import du fichier
On charge le fichier bitcoins.csv depuis le dossier `../data/`

In [71]:
fp = Path("../data/bitcoins.csv")
if not fp.exists():
    raise FileNotFoundError(
        f"Fichier introuvable :  {fp}. Vérifie que bitcoins.csv est dans ../data/"
    )
df = pd.read_csv(fp)

## ✅ Etape 1 — Volumétrie du dataset
**But :** Connaître immédiatement le nombre de lignes et de colonnes

In [72]:
df.shape

(24000, 7)

## ✅ Etape 2 — Types des colonnes (lecture Pandas)
**But :** Vérifier comment pandas interprète les types (et détecter rapidement les colonnes suspectes)

In [73]:
df.dtypes

Date           object
Open*         float64
High          float64
Low           float64
Close**       float64
Volume        float64
Market Cap    float64
dtype: object

## ✅ Etape 3 — Aperçu visuel du dataset (head, tail , sample)
**But :** Vérifier visuellement la cohérence des valeurs au début, la fin et au coeur du dataset <br />
`Selon le contexte, on peut utiliser tail() (CSV sales) ou sample() (dataset volumineux: df.sample(5, random_state=52)).`

In [74]:
df.head()

,Date,Open*,High,Low,Close**,Volume,Market Cap
0,Jan 01 2015,22534.95,23063.83,22296.16,22720.48,15569776.0,7.877863e+11
1,Jan 02 2015,57047.79,57254.07,56526.45,57015.36,41942747.0,4.085053e+11
2,Jan 03 2015,43946.44,44086.61,43093.73,43502.15,26950379.0,9.393686e+11
3,Jan 04 2015,35959.64,36686.08,35560.69,36378.15,43135834.0,9.657302e+11
4,Jan 05 2015,9445.52,NaN,9153.66,9925.85,40960650.0,1.349320e+12


In [75]:
df.tail()

,Date,Open*,High,Low,Close**,Volume,Market Cap
23995,Sep 11 2080,54806.95,55312.33,54481.67,NaN,13602611.0,6.640694e+11
23996,Sep 12 2080,25765.54,26086.96,24898.82,25341.29,36106450.0,1.864060e+12
23997,Sep 13 2080,22734.17,23166.90,22728.96,22752.43,44003147.0,1.760627e+12
23998,Sep 14 2080,13956.06,14386.30,13489.58,13543.53,27383880.0,5.868926e+11
23999,Sep 15 2080,38181.80,38399.30,37731.96,38018.86,42021486.0,1.473885e+11


In [76]:
df.sample(5, random_state=42)

,Date,Open*,High,Low,Close**,Volume,Market Cap
3111,Jul 09 2023,14597.12,15304.76,14564.78,15073.95,43883719.0,1.178662e+12
18679,Feb 21 2066,47889.08,47925.35,47775.76,47801.86,25383024.0,1.687627e+12
17472,Nov 02 2062,28553.50,28763.66,28386.50,28609.99,21186820.0,1.653921e+11
21451,Sep 24 2073,43752.81,44398.41,43539.36,44044.79,47424921.0,4.496947e+10
20800,Dec 13 2071,31559.84,32251.72,31387.61,32028.39,39867482.0,7.187313e+11


## ✅ Etape 4 — Valeurs manquantes
**But :** Quantifier les valeurs manquantes et identifier les colonnes à traiter en priorité

In [77]:
missing_count = df.isna().sum().sort_values(ascending=False)
missing_pct = (df.isna().mean() * 100).sort_values(ascending=False)
missing = pd.DataFrame({
     "missing_count": missing_count,
     "missing_pct": missing_pct.round(2)
})

missing.query("missing_count > 0")

,missing_count,missing_pct
Open*,720,3.0
Close**,720,3.0
High,480,2.0
Low,480,2.0
Volume,240,1.0
Market Cap,240,1.0
Date,120,0.5


## ✅ Etape 5 — Doublons
**But :** Détecter la présence des doublons sur les lignes complètes ou sur une c lé métier.

In [78]:
df.duplicated().sum()

np.int64(0)

### Variante — Doublons sur clé métier
A adapter selon le contexte métier (`id`, `timestamp`, `date`,ect.).

In [85]:
candidate_keys = ["id", "timestamp", "date", "Date"]
key_cols = [c for c in candidate_keys if c in df.columns]

if len(key_cols) > 0:
    df.duplicated(subset=key_cols).sum()
else:
    print("Aucune colonne clé standard trouvée. Définis explicitement ta clé métier.")

## 📌 À retenir

- **shape** : comprendre la volumétrie dès l’import
- **dtypes (ou info)** : détecter les types incohérents
- **head / tail / sample** : confirmer visuellement la cohérence
- **valeurs manquantes** : quantifier avant de décider (drop vs impute)
- **doublons** : contrôler sur lignes et clés métier

## 🔜 Suite logique du cookbook

Les étapes suivantes portent généralement sur :
- conversion des types
- traitement des valeurs manquantes
- nettoyage des chaînes de caractères
- contrôles qualité avancés

> 💡 Ces étapes sont aussi l’occasion de maîtriser des concepts transverses
> comme l’usage de `inplace`, la copie vs la vue, et l’impact mémoire.
